In [1]:
import pandas as pd
import sqlite3 as sql
import sqlalchemy as alc
import plotly.figure_factory as fig
import pyarrow.feather as feather
import utils.feather_cache as feather_cache
import seaborn as sns
import numpy as np
import display.heatmap as heatmap
import matplotlib.pyplot as plt
from data_types import tables
from views.algo_order_fills import display as algo_order_display
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from display import formatter

In [3]:
pd.options.display.max_rows = 50
pd.options.display.min_rows = 50

In [4]:
con = sql.connect('data/databases/sim_db/sim_db.sqlite')
run = pd.read_sql('select * from run', con)
log = pd.read_sql('select * from log', con)
# focus on runs w/ 5s to work
run = tables.runs_with_parameters(run, {'ms_to_work_shot': '5000'})
run_by_date = tables.demux_run_by_date(run)

In [5]:
feather.write_feather(log, 'data/cache/databases/log.feather')

In [7]:
def fmt_db(day):
    return f'data/310/2020-09-{day}/2020-09-{day}_310_24h.sqlite'

In [ ]:
books_map = {'2020-09-23': pd.read_sql('select * from books', sql.connect(fmt_db('09')) }

In [19]:
pcap_con = sql.connect('data/databases/')

,run_id,session_id,algo,begin_run_time,end_run_time,market_date,channel,base_algo_latency_ns,transaction_latency_ns,use_first_packet_st,...,param8,param9,param10,param11,param12,param13,param14,param15,ms_to_work_shot_is_match,all_params_match
1,2,0,Sniper,19:36:30.137,19:42:09.211,2020-09-10,310,22000,60000,1,...,min_value: 5,ms_to_work_shot: 5000,pay_up_ticks: 0,throttle_ns: 1000000,,,,,True,True
5,6,4,Sniper,19:42:31.398,19:45:39.511,2020-09-16,310,22000,60000,1,...,min_value: 5,ms_to_work_shot: 5000,pay_up_ticks: 0,throttle_ns: 1000000,,,,,True,True
9,10,8,Sniper,00:07:57.817,00:10:36.292,2020-09-07,310,22000,60000,1,...,min_value: 5,ms_to_work_shot: 5000,pay_up_ticks: 0,throttle_ns: 1000000,,,,,True,True
11,12,8,Sniper,00:07:57.817,00:09:49.257,2020-09-13,310,22000,60000,1,...,min_value: 5,ms_to_work_shot: 5000,pay_up_ticks: 0,throttle_ns: 1000000,,,,,True,True


In [ ]:
log = tables.LogData(log, run_by_date)
events = pd.read_feather('data/cache/310/2020-09-10/events.feather')
event_items = pd.read_feather('data/cache/310/2020-09-10/event_items.feather')
# securities
securities = pd.read_feather('data/cache/310/2020-09-10/securities.feather')
securities = tables.SecuritiesData(securities)
sec_books = {'2020-09-10': {
      securities.id_for_symbol('ESU0') : feather.read_feather('data/cache/310/2020-09-10/ESU0_books.feather'),
      securities.id_for_symbol('ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESZ0_books.feather'),
      securities.id_for_symbol('ESU0-ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESZ0_books.feather'),
      securities.id_for_symbol('ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU1_books.feather'),
      securities.id_for_symbol('ESU0-ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESU1_books.feather')
   },
   '2020-09-13': {
      securities.id_for_symbol('ESU0') : feather.read_feather('data/cache/310/2020-09-10/ESU0_books.feather'),
      securities.id_for_symbol('ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESZ0_books.feather'),
      securities.id_for_symbol('ESU0-ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESZ0_books.feather'),
      securities.id_for_symbol('ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU1_books.feather'),
      securities.id_for_symbol('ESU0-ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESU1_books.feather')
   },
   '2020-09-16': {
      securities.id_for_symbol('ESU0') : feather.read_feather('data/cache/310/2020-09-10/ESU0_books.feather'),
      securities.id_for_symbol('ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESZ0_books.feather'),
      securities.id_for_symbol('ESU0-ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESZ0_books.feather'),
      securities.id_for_symbol('ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU1_books.feather'),
      securities.id_for_symbol('ESU0-ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESU1_books.feather')
   }
}
# book stats
book_stats = feather.read_feather('data/cache/310/2020-09-10/book_stats.feather')
book_stats = tables.BookStatsData(sec_books, book_stats, events)

In [4]:
log = feather.read_feather('data/cache/databases/sim_db/log.feather')
log = tables.LogData(log, run_by_date)
events = pd.read_feather('data/cache/310/2020-09-10/events.feather')
event_items = pd.read_feather('data/cache/310/2020-09-10/event_items.feather')
# securities
securities = pd.read_feather('data/cache/310/2020-09-10/securities.feather')
securities = tables.SecuritiesData(securities)
sec_books = {
   securities.id_for_symbol('ESU0') : feather.read_feather('data/cache/310/2020-09-10/ESU0_books.feather'),
   securities.id_for_symbol('ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESZ0_books.feather'),
   securities.id_for_symbol('ESU0-ESZ0'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESZ0_books.feather'),
   securities.id_for_symbol('ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU1_books.feather'),
   securities.id_for_symbol('ESU0-ESU1'): feather.read_feather('data/cache/310/2020-09-10/ESU0-ESU1_books.feather')
}
# book stats
book_stats = feather.read_feather('data/cache/310/2020-09-10/book_stats.feather')
book_stats = tables.BookStatsData(sec_books, book_stats, events)

In [32]:
run_map = {
    '2020-09-10': {
        'ESU0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-10', 'ESU0', 2),
        'ESZ0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-10', 'ESZ0', 2),
        'ESU1': tables.merge_run_with_books(log, securities, book_stats, '2020-09-10', 'ESU1', 2)
    },
    '2020-09-13': {
        'ESU0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-13', 'ESU0', 12),
        'ESZ0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-13', 'ESZ0', 12),
        'ESU1': tables.merge_run_with_books(log, securities, book_stats, '2020-09-13', 'ESU1', 12)
    },
    '2020-09-16': {
        'ESU0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-16', 'ESU0', 6),
        'ESZ0': tables.merge_run_with_books(log, securities, book_stats, '2020-09-16', 'ESZ0', 6),
        'ESU1': tables.merge_run_with_books(log, securities, book_stats, '2020-09-16', 'ESU1', 6)
    }
}

In [135]:
desc_cols = ['run_id', 'side', 'price_order', 'price_fill', 'fill_profit_delta', 'fill_t_timestamp_delta']
var_side_cols = ['avg_price_0_var_1s', 'variance_0', 'avg_price_delta', 'avg_price_0']
var_bid_ask_cols = [f'bid_{c}' for c in var_side_cols] + [f'ask_{var_side_cols[i]}' for i in range(len(var_side_cols) - 1, -1)]
std_side_cols = ['avg_price_0_std_1s', 'variance_0', 'avg_price_delta', 'avg_price_0']
std_bid_ask_cols = [f'bid_{c}' for c in std_side_cols] + [f'ask_{std_side_cols[i]}' for i in range(len(std_side_cols) - 1, -1)]

In [7]:
formatter.print_pretty(log.fills).apply(lambda x: heatmap.gradient_color_list(x, [(0,0,1),(0,0,0),(1,0,0)]), subset=['fill_profit_delta'])
#display.style.highlight_null().render().split('\n')[:10]
#display.style.apply(lambda c: heatmap.gradient_color_list(c, [(0,0,1),(0,0,0),(1,0,0)]), subset=['fill_price_delta'])
#display.head(2).to_html()

,run_id,ok,ev_eid1_order,fill_ev_eid_delta,fill_t_timestamp_delta,side,price_order,price_fill,fill_price_delta,fill_profit_delta,qty_order,qty_fill
138,2,18446744073709551582,30553,165,5.04,SIDE_SELL,333700.00,333675.00,-25.00,-25.000000,1,1
144,1,18446744073709551582,30553,142,2.24,SIDE_SELL,333700.00,333675.00,-25.00,-25.000000,1,1
294,1,18446744073709551534,94516,321,1.30,SIDE_SELL,333950.00,333875.00,-75.00,-75.000000,1,1
312,2,18446744073709551534,94516,573,5.08,SIDE_SELL,333950.00,333875.00,-75.00,-75.000000,1,1
324,3,18446744073709551534,94516,633,10.69,SIDE_SELL,333950.00,333850.00,-100.00,-100.000000,1,1
369,4,18446744073709551534,94516,916,30.26,SIDE_SELL,333950.00,333850.00,-100.00,-100.000000,1,1
381,1,18446744073709551519,251014,394,1.00,SIDE_SELL,334200.00,334150.00,-50.00,-50.000000,1,1
387,2,18446744073709551519,251014,688,5.09,SIDE_SELL,334200.00,334125.00,-75.00,-75.000000,1,1
390,3,18446744073709551519,251014,1233,10.03,SIDE_SELL,334200.00,334100.00,-100.00,-100.000000,1,1
393,4,18446744073709551519,251014,2024,30.02,SIDE_SELL,334200.00,334050.00,-150.00,-150.000000,1,1


In [33]:
# bc i hate looking at -0.0
def no_neg_zero(val):
    if val == 0:
        if isinstance(val, float):
            return 0.0
        else: return 0
    return val

In [8]:
def bbo_cols():
    return ['bid_q_0', 'bid_p_0', 'ask_p_0', 'ask_q_0']

def book_stat_bbo_cols():
    return ['bid_price_0', 'ask_price_0']

def bbo_avg_cols(secs = [1,5]):
    rv = []
    for c in book_stat_bbo_cols():
        for s in secs:
            rv.append(f'{c}_avg_{s}s')
    return rv

def bbo_var_cols(secs = [1,5]):
    #return [f'{c}_var_{secs}s' for c in book_stat_bbo_cols() for sec in secs]
    rv = []
    for c in book_stat_bbo_cols():
        for s in secs:
            rv.append(f'{c}_var_{s}s')
    return rv

def sample_limit(df: pd.DataFrame, max_rows = 25):
    if len(df) > max_rows:
        return df.sample(max_rows)
    return df

def print_pretty(df: pd.DataFrame, max_rows = 25, print_row_count = False):
    col_styles = {}
    for c in df.columns:
       if 'price' in c or 'avg' in c or 'profit' in c or 'bid_p' in c or 'ask_p' in c:
          col_styles[c] = "{:.2f}"
       elif 'timestamp' in c:
          col_styles[c] = lambda t: f'{t.total_seconds():.2f}'

    for c in df.columns:
       if 'price' in c or 'avg' in c or 'profit' in c or 'var' in c:
          col_styles[c] = "{:.2f}"
       elif 'timestamp' in c:
          col_styles[c] = lambda t: f'{t.total_seconds():.2f}'
    if print_row_count:
        print(f'Total Rows: {len(df)} (sample {max_rows}).')
    df = sample_limit(df)
    colset = list(set(['fill_profit_delta'] + tables.bbo_var_cols(1) + tables.bbo_var_inter_period_delta(1, 5)).intersection(set([c.split(' ')[0] for c in df.columns])))
    print(f"set is {colset}")
    return df.style.format(col_styles).set_table_styles([
       dict(selector='tr', props=[('text-align', 'right')])
       #dict(selector='tr', props=[('background', '#88ff0000')])
    ]).background_gradient(cmap=LinearSegmentedColormap.from_list('', [(0,0,1), (1,1,1), (1,0,0)]), subset=colset)
    #.background_gradient(cmap=LinearSegmentedColormap.from_list('', [(0,0,1), (1,1,1), (1, 0, 0)]), subset=['bid_p_0_var_1s'])

In [21]:
increasing_variance = sweetness.copy()
increasing_variance['bid_price_0_avg_1s_delta'] = increasing_variance['bid_price_0'] - increasing_variance['bid_price_0_avg_1s']
increasing_variance['bid_price_0_avg_5s_delta'] = increasing_variance['bid_price_0'] - increasing_variance['bid_price_0_avg_5s']
increasing_variance.sort_values('fill_profit_delta')

,run_id,log_entry_order_order,sim_seq_order,t_time_order,ev_eid1_order,ev_eid2_order,ev_t_time_order,ev_o_time_order,opp_id,change_type,...,ask_price_0_std_5s_delta,ask_price_0_var_1s_delta,ask_price_0_var_5s_delta,ask_avg_price_0_avg_1s_delta,ask_avg_price_0_avg_5s_delta,ask_avg_price_0_std_1s_delta,ask_avg_price_0_std_5s_delta,ask_avg_price_0_var_1s_delta,ask_avg_price_0_var_5s_delta,cross_bbo_delta
43,2,548044,270,2020-09-11 14:11:03.292763007,7217515,2,1599833463292414544,1599833463292763007,1599833463292414544,added,...,0.002319,0.054013,0.361974,-0.011920,0.011563,0.004006,0.001490,0.206044,0.231754,25.0
71,2,1185722,472,2020-09-11 15:51:25.933838066,12048610,2,1599839485931985481,1599839485933838066,1599839485931985481,added,...,-0.048905,-20.694756,-2.398209,-0.310937,-0.040465,-0.746292,-0.047437,-18.230580,-2.141229,25.0
41,2,495222,267,2020-09-11 14:02:03.505173722,6545472,2,1599832923502677371,1599832923505173722,1599832923502677371,added,...,-0.051478,-8.393941,-3.913760,-0.110198,-0.036824,-0.129714,-0.048225,-7.208589,-3.585242,25.0
38,2,325646,248,2020-09-11 13:51:48.799543539,5664536,2,1599832308798551691,1599832308799543539,1599832308798551691,added,...,0.007132,-2.383464,0.446232,-0.095981,0.008682,-0.074626,0.006225,-1.745728,0.406640,25.0
93,2,1616340,593,2020-09-11 17:19:13.704715743,14963981,2,1599844753702936427,1599844753704715743,1599844753702936427,added,...,-0.031178,-1.087425,-3.909214,-0.045812,-0.039180,-0.036020,-0.030110,-0.866012,-3.696713,25.0
44,2,647186,281,2020-09-11 14:15:26.570434699,7588560,2,1599833726568612654,1599833726570434699,1599833726568612654,added,...,-0.015429,-2.430517,-1.891277,-0.073501,-0.029563,-0.076432,-0.012787,-2.254875,-1.572140,25.0
39,2,459540,260,2020-09-11 13:57:58.617524969,6187272,2,1599832678616776383,1599832678617524969,1599832678616776383,added,...,0.029536,-1.742796,2.853328,0.051516,0.037871,-0.046284,0.024462,-2.641211,2.371216,25.0
95,2,1708158,607,2020-09-11 18:13:14.344816190,16788491,2,1599847994342842492,1599847994344816190,1599847994342842492,added,...,-0.014887,-0.286988,-1.326593,-0.025710,-0.031892,0.002421,-0.011759,0.077462,-1.053981,25.0
98,2,1762680,611,2020-09-11 18:36:20.922534614,17414907,2,1599849380919442346,1599849380922534614,1599849380919442346,added,...,-0.218151,-29.352722,-7.224033,-0.360368,-0.078301,-1.941708,-0.254039,-26.841417,-6.420167,50.0
101,2,1858654,627,2020-09-11 18:58:02.570393329,18091486,2,1599850682569758818,1599850682570393329,1599850682569758818,added,...,-0.059170,-2.944561,-4.614442,-0.099108,-0.062533,-0.058768,-0.049361,-2.017195,-3.779087,25.0


In [35]:
def get_run_covs(date: str, sym: str) -> pd.DataFrame:
    tmp = run_map[date][sym].copy()
    tmp['bbo_cross_delta'] = tmp['ask_price_0'] - tmp['bid_price_0']
    tmp = tmp.drop(columns=['run_id', 'fill_ev_eid_delta', 'ok', 'side', 'price_order', 'price_fill', 'qty_order', 'qty_fill', 'ask_price_0', 'bid_price_0'])
    print(len(tmp))
    tmp = tmp[['fill_profit_delta'] + bbo_avg_cols() + bbo_var_cols() + ['bbo_cross_delta', 'bid_price_0_var_1_5s_delta', 'ask_price_0_var_1_5s_delta']]
    return tmp.cov().sort_values(by='fill_profit_delta')

In [36]:
print_pretty(get_run_covs('2020-09-10', 'ESU0'))

835
set is ['fill_profit_delta']


,fill_profit_delta,bid_price_0_avg_1s,bid_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_avg_5s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_var_1s,ask_price_0_var_5s,bbo_cross_delta,bid_price_0_var_1_5s_delta,ask_price_0_var_1_5s_delta
ask_price_0_var_5s,-2468.84,-433596.73,-408643.40,-430750.22,-406510.71,1216436.99,4222608.07,1175795.36,4201804.93,2781.376547,-3009402.32,-3029228.96
bid_price_0_var_5s,-2307.73,-448824.61,-422203.54,-445893.77,-419953.62,1263786.78,4360229.35,1168933.36,4222608.07,2852.757808,-3099825.80,-3056905.95
bbo_cross_delta,-3.28,-1200.69,-1196.02,-1192.00,-1189.88,1684.68,2852.76,1539.33,2781.38,91.444808,-1167.04,-1240.95
fill_profit_delta,1024.12,2120.80,2164.45,2122.20,2164.79,1517.24,-2307.73,1521.68,-2468.84,-3.280310,3817.07,3982.88
bid_price_0_var_1s,1517.24,-99118.01,-89503.73,-97913.91,-88730.72,973750.75,1263786.78,923884.23,1216436.99,1684.680607,-290036.03,-292552.76
ask_price_0_var_1s,1521.68,-89037.71,-80457.67,-87784.43,-79675.66,923884.23,1168933.36,917545.21,1175795.36,1539.329781,-245049.12,-258250.15
bid_price_0_avg_1s,2120.80,1386690.83,1385250.45,1386165.74,1384849.22,-99118.01,-448824.61,-89037.71,-433596.73,-1200.692530,347728.23,342580.33
ask_price_0_avg_1s,2122.20,1386165.74,1384738.48,1385658.43,1384349.21,-97913.91,-445893.77,-87784.43,-430750.22,-1192.000896,346002.58,340988.27
bid_price_0_avg_5s,2164.45,1385250.45,1384928.87,1384738.48,1384537.77,-89503.73,-422203.54,-80457.67,-408643.40,-1196.018933,330620.67,326108.77
ask_price_0_avg_5s,2164.79,1384849.22,1384537.77,1384349.21,1384157.42,-88730.72,-419953.62,-79675.66,-406510.71,-1189.881316,329145.17,324759.61


In [37]:
print_pretty(get_run_covs('2020-09-13', 'ESU0'))

339
set is ['fill_profit_delta']


,fill_profit_delta,bid_price_0_avg_1s,bid_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_avg_5s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_var_1s,ask_price_0_var_5s,bbo_cross_delta,bid_price_0_var_1_5s_delta,ask_price_0_var_1_5s_delta
bid_price_0_var_5s,-3497.89,-197362.28,-198410.48,-194602.05,-196065.21,493882.70,1270770.88,343009.70,1035278.20,2686.600808,-776888.19,-692268.51
ask_price_0_avg_5s,-2743.85,763480.18,763219.80,762386.36,762139.02,-58847.82,-196065.21,-50849.85,-175764.54,-1494.721331,137217.40,124914.69
ask_price_0_avg_1s,-2722.22,764322.82,763473.71,763226.84,762386.36,-59204.22,-194602.05,-50828.67,-173682.96,-1477.403520,135397.84,122854.29
bid_price_0_avg_5s,-2701.27,764600.63,764335.45,763473.71,763219.80,-60442.18,-198410.48,-51832.88,-177056.55,-1533.680572,137968.30,125223.68
bid_price_0_avg_1s,-2678.57,765460.87,764600.63,764322.82,763480.18,-61187.11,-197362.28,-52028.30,-175231.45,-1522.151173,136175.18,123203.15
bid_price_0_var_1s,-2471.03,-61187.11,-60442.18,-59204.22,-58847.82,347572.94,493882.70,245830.62,347807.52,1371.893836,-146309.76,-101976.91
ask_price_0_var_1s,-1058.93,-52028.30,-51832.88,-50828.67,-50849.85,245830.62,343009.70,191100.66,262664.39,738.327459,-97179.08,-71563.73
ask_price_0_var_5s,-823.62,-175231.45,-177056.55,-173682.96,-175764.54,347807.52,1035278.20,262664.39,906124.26,1343.775775,-687470.68,-643459.87
ask_price_0_var_1_5s_delta,-235.31,123203.15,125223.68,122854.29,124914.69,-101976.91,-692268.51,-71563.73,-643459.87,-605.448316,590291.60,571896.15
bbo_cross_delta,-107.16,-1522.15,-1533.68,-1477.40,-1494.72,1371.89,2686.60,738.33,1343.78,148.168997,-1314.71,-605.45


In [38]:
print_pretty(get_run_covs('2020-09-16', 'ESU0'))


362
set is ['fill_profit_delta']


,fill_profit_delta,bid_price_0_avg_1s,bid_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_avg_5s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_var_1s,ask_price_0_var_5s,bbo_cross_delta,bid_price_0_var_1_5s_delta,ask_price_0_var_1_5s_delta
bid_price_0_var_1_5s_delta,-1637.06,196934.02,195306.39,195660.49,194328.13,-110639.02,-1438591.24,-129838.16,-1377335.77,22.141074,1327952.22,1247497.61
ask_price_0_var_1_5s_delta,-1255.11,183447.30,182723.92,183513.20,182715.43,-98860.20,-1346357.81,-100608.50,-1341880.64,621.632171,1247497.61,1241272.14
bbo_cross_delta,-34.74,-272.45,-274.23,-246.88,-259.34,10.63,-3.89,119.41,-493.36,71.203379,22.14,621.63
ask_price_0_var_1s,465.34,-112830.35,-109946.11,-112246.74,-109512.74,214000.48,343838.64,223778.08,324386.58,119.414252,-129838.16,-100608.50
bid_price_0_var_1s,566.77,-106509.56,-104080.92,-106208.37,-103851.30,211176.56,321815.58,214000.48,312860.68,10.630161,-110639.02,-98860.20
ask_price_0_var_5s,1669.83,-297175.84,-293575.85,-296651.21,-293127.44,312860.68,1687938.49,324386.58,1664035.91,-493.361966,-1377335.77,-1341880.64
fill_profit_delta,1748.83,7014.43,7049.90,7013.13,7052.33,566.77,2151.11,465.34,1669.83,-34.740821,-1637.06,-1255.11
bid_price_0_var_5s,2151.11,-304350.50,-300303.06,-302771.65,-299090.63,321815.58,1757994.68,343838.64,1687938.49,-3.890159,-1438591.24,-1346357.81
ask_price_0_avg_1s,7013.13,1534969.07,1532477.82,1534675.31,1532144.07,-106208.37,-302771.65,-112246.74,-296651.21,-246.884887,195660.49,183513.20
bid_price_0_avg_1s,7014.43,1535302.83,1532797.90,1534969.07,1532438.24,-106509.56,-304350.50,-112830.35,-297175.84,-272.446475,196934.02,183447.30


In [10]:
tmp = sample_z0.copy()
tmp['bbo_cross_delta'] = tmp['ask_price_0'] - tmp['bid_price_0']
tmp = tmp.drop(columns=['run_id', 'fill_ev_eid_delta', 'ok', 'side', 'price_order', 'price_fill', 'qty_order', 'qty_fill', 'ask_price_0', 'bid_price_0'])
print(len(tmp))
tmp = tmp[['fill_profit_delta'] + bbo_avg_cols() + bbo_var_cols() + ['bbo_cross_delta']]
print_pretty(tmp.cov().sort_values(by='fill_profit_delta'))

871
set is ['fill_profit_delta']


,fill_profit_delta,bid_price_0_avg_1s,bid_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_avg_5s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_var_1s,ask_price_0_var_5s,bbo_cross_delta
ask_price_0_avg_1s,-1852.64,1501896.80,1498753.04,1501740.05,1498494.30,-196771.03,-497306.26,-199017.43,-488841.51,-431.693103
bid_price_0_avg_1s,-1841.13,1502088.89,1498930.52,1501896.80,1498644.38,-198379.03,-499206.28,-200880.56,-491133.51,-455.899214
ask_price_0_avg_5s,-1829.66,1498644.38,1496586.15,1498494.30,1496331.15,-189981.42,-474139.99,-194066.37,-467289.20,-483.024218
bid_price_0_avg_5s,-1828.23,1498930.52,1496867.53,1498753.04,1496586.15,-190779.65,-475392.16,-195040.80,-468914.40,-501.326628
bbo_cross_delta,5.45,-455.90,-501.33,-431.69,-483.02,481.03,1511.73,1161.98,2034.80,139.204508
bid_price_0_var_1s,201.79,-198379.03,-190779.65,-196771.03,-189981.42,815375.10,1033822.36,793945.52,996178.41,481.025761
ask_price_0_var_1s,580.59,-200880.56,-195040.80,-199017.43,-194066.37,793945.52,943208.60,851890.20,959138.38,1161.977746
fill_profit_delta,878.56,-1841.13,-1828.23,-1852.64,-1829.66,201.79,2644.14,580.59,3290.64,5.446079
bid_price_0_var_5s,2644.14,-499206.28,-475392.16,-497306.26,-474139.99,1033822.36,3359778.95,943208.60,3263941.22,1511.734518
ask_price_0_var_5s,3290.64,-491133.51,-468914.40,-488841.51,-467289.20,996178.41,3263941.22,959138.38,3237017.06,2034.803295


In [75]:
tmp = display.copy()
tmp['bbo_cross_delta'] = tmp['ask_price_0'] - tmp['bid_price_0']
tmp = tmp.drop(columns=['run_id', 'fill_ev_eid_delta', 'ok', 'price_order', 'price_fill', 'qty_order', 'qty_fill', 'ask_price_0', 'bid_price_0'])
print(len(tmp))
print_pretty(tmp.cov().sort_values(by='fill_profit_delta'))

134
set is ['fill_profit_delta']


,fill_price_delta,fill_profit_delta,rgtm,ask_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_var_1s,ask_price_0_var_5s,bid_price_0_avg_5s,bid_price_0_avg_1s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_avg_1s_delta,ask_price_0_avg_5s_delta,bid_price_0_avg_1s_delta,bid_price_0_avg_5s_delta,bid_price_0_center_product,ask_price_0_center_product,bid_price_0_var_1s_5s_delta,ask_price_0_var_1s_5s_delta,bbo_cross_delta
bid_price_0_var_5s,10492.41,-10492.41,320.028593,-397264.57,-392044.07,1369950.47,4490899.59,-399347.82,-395643.80,1362206.37,4303591.53,-14710.91,-9490.41,-13300.84,-9596.82,4564581.35,4361319.31,-2941385.15,-3120949.12,2189.663262
ask_price_0_var_5s,8396.68,-8396.68,354.274031,-357949.82,-347949.63,1586305.67,4980095.91,-361543.09,-354017.20,1449805.72,4490899.59,-13693.16,-3692.96,-11760.64,-4234.75,4359652.08,4168184.09,-3041093.87,-3393790.24,4135.046806
fill_price_delta,4653.83,-4653.83,-45.676692,-16998.93,-17076.80,-6483.94,8396.68,-16987.76,-17059.59,-6021.51,10492.41,417.27,339.39,466.54,394.72,-26244.32,-18184.82,-16513.92,-14880.62,-66.490854
bid_price_0_avg_1s_delta,466.54,-466.54,-6.601167,235.73,707.09,-7669.68,-11760.64,245.39,720.73,-9486.58,-13300.84,826.71,1298.07,958.00,1433.34,-35782.18,-23956.10,3814.26,4090.96,-144.931686
ask_price_0_avg_1s_delta,417.27,-417.27,-5.710924,-877.61,-418.42,-8334.86,-13693.16,-866.88,-398.21,-9768.29,-14710.91,805.02,1264.21,826.71,1295.38,-30499.71,-22687.90,4942.63,5358.30,-41.900499
bid_price_0_avg_5s_delta,394.72,-394.72,-5.600882,-1285.38,549.07,-10343.33,-4234.75,-1278.64,553.66,-13986.98,-9596.82,1295.38,3129.83,1433.34,3265.63,-43405.07,-28541.26,-4390.16,-6108.58,-142.547532
ask_price_0_avg_5s_delta,339.39,-339.39,-4.580514,-2399.82,-563.95,-9830.84,-3692.96,-2396.27,-564.51,-13455.87,-9490.41,1264.21,3100.09,1298.07,3129.83,-37804.98,-27165.87,-3965.46,-6137.87,-33.292848
rgtm,-45.68,45.68,1.007519,-23.61,-22.48,201.57,354.27,-23.72,-22.72,198.16,320.03,-5.71,-4.58,-6.60,-5.60,693.46,537.64,-121.87,-152.70,1.127820
bbo_cross_delta,-66.49,66.49,1.127820,-1635.15,-1626.54,2170.71,4135.05,-1652.57,-1650.18,1643.07,2189.66,-41.90,-33.29,-144.93,-142.55,5383.03,1002.77,-546.59,-1964.34,126.669285
fill_profit_delta,-4653.83,4653.83,45.676692,16998.93,17076.80,6483.94,-8396.68,16987.76,17059.59,6021.51,-10492.41,-417.27,-339.39,-466.54,-394.72,26244.32,18184.82,16513.92,14880.62,66.490854


In [205]:
tmp = display.copy()
tmp['bbo_cross_delta'] = tmp['ask_price_0'] - tmp['bid_price_0']
tmp = tmp.drop(columns=['run_id', 'ok', 'side', 'price_order', 'price_fill', 'qty_order', 'qty_fill', 'ask_price_0', 'bid_price_0'])
print(len(tmp))
print_pretty(tmp.cov().sort_values(by='fill_profit_delta'))

18
set is ['fill_profit_delta']


,fill_ev_eid_delta,fill_price_delta,fill_profit_delta,ask_price_0_avg_5s,ask_price_0_avg_1s,ask_price_0_var_1s,ask_price_0_var_5s,bid_price_0_avg_5s,bid_price_0_avg_1s,bid_price_0_var_1s,bid_price_0_var_5s,ask_price_0_avg_1s_delta,ask_price_0_avg_5s_delta,bid_price_0_avg_1s_delta,bid_price_0_avg_5s_delta,bid_price_0_center_product,ask_price_0_center_product,bid_price_0_var_1s_5s_delta,ask_price_0_var_1s_5s_delta,bbo_cross_delta
bid_price_0_center_product,-789118.767289,62513.47,-62513.47,3848532.14,3878286.30,-10198057.29,-8806170.94,3853739.42,3883836.39,-10246896.66,-9112610.43,-104283.78,-74529.62,-63633.49,-33536.52,46473742.89,36557863.59,-1134286.24,-1391886.35,-46200.376458
ask_price_0_center_product,-2299797.096479,58096.38,-58096.38,2476248.65,2464370.67,-8348438.16,-5808561.96,2480551.19,2468853.81,-7865637.21,-5771173.78,-179030.42,-190908.40,-157812.30,-169509.68,36557863.59,35315728.57,-2094463.43,-2539876.20,-25701.256814
bid_price_0_avg_1s,34909.926844,18270.89,-18270.89,1738376.14,1749185.09,-1275175.84,-1119074.80,1739425.97,1750409.43,-1302294.38,-1182363.43,-3770.11,7038.84,4744.51,15727.97,3883836.39,2468853.81,-119930.96,-156101.03,-9738.956695
ask_price_0_avg_1s,36020.807477,18245.73,-18245.73,1737179.19,1747964.78,-1272563.61,-1117140.40,1738226.93,1749185.09,-1299749.53,-1180419.01,-3765.73,7019.86,4739.71,15697.87,3878286.30,2464370.67,-119330.53,-155423.20,-9725.747693
bid_price_0_avg_5s,37152.074274,18137.63,-18137.63,1728209.07,1738226.93,-1247174.03,-1090438.15,1729246.30,1739425.97,-1270370.85,-1151153.54,-4700.91,5316.95,3590.18,13769.85,3853739.42,2480551.19,-119217.31,-156735.89,-9490.133137
ask_price_0_avg_5s,37449.809059,18115.32,-18115.32,1727173.46,1737179.19,-1245380.14,-1089035.19,1728209.07,1738376.14,-1268600.30,-1149724.22,-4694.62,5311.11,3590.89,13757.97,3848532.14,2476248.65,-118876.08,-156344.95,-9482.461974
fill_ev_eid_delta,1832223.271242,3745.51,-3745.51,37449.81,36020.81,1610860.84,640171.48,37152.07,34909.93,1386937.72,557658.43,34181.81,32752.81,30857.97,28615.82,-789118.77,-2299797.10,829279.30,970689.36,4434.722222
fill_price_delta,3745.506536,892.57,-892.57,18115.32,18245.73,-21905.02,-16293.43,18137.63,18270.89,-21732.76,-16571.10,-79.88,50.53,7.30,140.56,62513.47,58096.38,-5161.65,-5611.59,-112.336601
bid_price_0_avg_5s_delta,28615.817883,140.56,-140.56,13757.97,15697.87,-21184.82,-46193.27,13769.85,15727.97,-36582.03,-54203.66,3778.56,5718.47,4215.44,6173.56,-33536.52,-169509.68,17621.63,25008.45,-466.974706
ask_price_0_avg_5s_delta,32752.805320,50.53,-50.53,5311.11,7019.86,-838.51,-29062.64,5316.95,7038.84,-15309.48,-36106.65,3747.20,5455.95,3996.58,5718.47,-74529.62,-190908.40,20797.17,28224.13,-268.355020


In [46]:
sweetness

,run_id,log_entry_order_order,sim_seq_order,t_time_order,ev_eid2_order,ev_t_time_order,ev_o_time_order,opp_id,change_type,symbol_x,...,ask_avg_price_0_var_1s_delta,ask_avg_price_0_var_5s_delta,cross_bbo_delta,eid,sid_x_y,symbol,bid_q_0_y,bid_p_0_y,ask_p_0_y,ask_q_0_y
9334838,2,720,7,2020-09-10 22:01:00.666739805,2,1599775260666256607,1599775260666739805,1599775260666256607,added,ESU0,...,-0.072120,1.100954,50.0,0,NaN,NaN,NaN,NaN,NaN,NaN
9334838,2,2592,12,2020-09-10 22:04:47.000864711,2,1599775487000391585,1599775487000864711,1599775487000391585,added,ESU0,...,2.037058,-3.591271,25.0,1,NaN,NaN,NaN,NaN,NaN,NaN
9334838,2,3430,16,2020-09-10 22:08:19.142422492,2,1599775699142041525,1599775699142422492,1599775699142041525,added,ESU0,...,-1.201840,-1.533540,25.0,2,NaN,NaN,NaN,NaN,NaN,NaN
0,2,4492,18,2020-09-10 22:13:01.073111994,2,1599775981072740507,1599775981073111994,1599775981072740507,added,ESU0,...,0.000000,3.565993,50.0,3,12181.0,ESU0,1.0,334100.0,334200.0,2.0
1,2,6093,20,2020-09-10 22:15:02.976199587,2,1599776102975257570,1599776102976199587,1599776102975257570,added,ESU0,...,4.526586,-3.565426,25.0,4,12181.0,ESU0,1.0,334100.0,334200.0,2.0
1,2,7316,25,2020-09-10 22:20:30.234385533,2,1599776430233531814,1599776430234385533,1599776430233531814,added,ESU0,...,-5.447903,2.913859,25.0,5,12181.0,ESU0,1.0,334100.0,334200.0,2.0
2,2,8293,27,2020-09-10 22:24:06.885906100,2,1599776646885526722,1599776646885906100,1599776646885526722,added,ESU0,...,-5.315152,-0.419067,50.0,6,12181.0,ESU0,1.0,334100.0,334200.0,2.0
2,2,10128,30,2020-09-10 22:38:18.208658881,2,1599777498208533046,1599777498208658881,1599777498208533046,added,ESU0,...,0.616162,3.225758,50.0,7,12181.0,ESU0,1.0,334100.0,334200.0,2.0
3,2,12509,34,2020-09-11 00:15:39.416716315,2,1599783339415862869,1599783339416716315,1599783339415862869,added,ESU0,...,-5.269354,1.683194,25.0,8,12181.0,ESU0,1.0,334100.0,334200.0,2.0
3,2,16926,35,2020-09-11 00:19:29.663924587,2,1599783569660144380,1599783569663924587,1599783569660144380,added,ESU0,...,17.839090,14.739335,75.0,9,12181.0,ESU0,1.0,334100.0,334200.0,2.0


In [31]:
sweetness.sort_values(by='fill_profit_delta')

col_styles = {}
for c in sweetness.columns:
   if 'price' in c or 'avg' in c or 'profit' in c or 'bid_p' in c or 'ask_p' in c:
      col_styles[c] = "{:.2f}"
   elif 'time' in c:
      col_styles[c] = lambda t: f'{t.total_seconds():.2f}'

display = sweetness[['run_id', 'ok', 'fill_ev_eid_delta', 'fill_t_time_delta',
   'side', 'price_order', 'price_fill', 'fill_price_delta', 'fill_profit_delta', 'rgtm', 'qty_order', 'qty_fill'] +
    ['ask_price_0_avg_5s', 'ask_price_0_avg_1s', 'ask_price_0', 'ask_price_0_var_1s', 'ask_price_0_var_5s'] +
    ['bid_price_0_avg_5s', 'bid_price_0_avg_1s', 'bid_price_0', 'bid_price_0_var_1s', 'bid_price_0_var_5s']]
    #bbo_var_cols(1) + bbo_var_cols(5) + bbo_avg_cols(1) + bbo_avg_cols(5)]
display['ask_price_0_avg_1s_delta'] = display['ask_price_0'] - display['ask_price_0_avg_1s']
display['ask_price_0_avg_5s_delta'] = display['ask_price_0'] - display['ask_price_0_avg_5s']
display['bid_price_0_avg_1s_delta'] = display['bid_price_0'] - display['bid_price_0_avg_1s']
display['bid_price_0_avg_5s_delta'] = display['bid_price_0'] - display['bid_price_0_avg_5s']
#display['bid_price_0_center_product_diff'] = (display['bid_price_0'] - display['bid_price_0_avg_5s_delta']) * (display['bid_price_0_avg_1s_delta'] = display['bid_price_0'])
display['bid_price_0_center_product'] = display['bid_price_0_avg_1s_delta'] * display['bid_price_0_avg_5s_delta']
display['ask_price_0_center_product'] = display['ask_price_0_avg_1s_delta'] * display['ask_price_0_avg_5s_delta']

# display data summary
max_disp_rows = 25
print(f"Total Rows (before): {len(display)}")
print("-----------------------------------------------")
num_missed_before = len(display.loc[display.fill_profit_delta < -1])
total_rows_before = len(display)
print(f"Num bad fills (before): {num_missed_before}")
print(f"Avg fill profit delta (before): {display['fill_profit_delta'].mean():.{2}f}")
print(f"Mode fill profit delta (before): {no_neg_zero(display['fill_profit_delta'].mode()[0]):.{2}f}")
print(f".75 Quantile fill profit delta (before): {display['fill_profit_delta'].quantile(0.25):.{2}f}")
print(f".80 Quantile fill profit delta (before): {display['fill_profit_delta'].quantile(0.20):.{2}f}")
print(f".85 Quantile fill profit delta (before): {display['fill_profit_delta'].quantile(0.15):.{2}f}")
print(f".90 Quantile fill profit delta (before): {display['fill_profit_delta'].quantile(0.10):.{2}f}")
print("-----------------------------------------------")

# filter by stats that may be connected to fill profit deltas
#display = display.loc[(display.bid_p_0_var_1s > (display.bid_p_0_var_5s)) & (display.side == 'SIDE_BUY')]
display['bid_price_0_var_1s_5s_delta'] = display['bid_price_0_var_1s'] - display['bid_price_0_var_5s']
display['ask_price_0_var_1s_5s_delta'] = display['ask_price_0_var_1s'] - display['ask_price_0_var_5s']
#display = display.loc[display.bid_price_0_var_1s_5s_delta > 10]
#display = display.loc[display.bid_price_0_var_1s_5s_delta > 10]
#display = display.loc[display.bid_price_0_var_1s_5s_delta > 10]
#display = display.loc[display.bid_price_0_center_product > 100]
num_missed_after = len(display.loc[display.fill_profit_delta < -1])
total_rows_after = len(display)
print(f"Total Rows (after): {len(display)}.")
print(f"Num bad fills (after): {num_missed_after}")
print(f"Bad fill percentage changed from {(num_missed_before / total_rows_before):.{3}f} to {(num_missed_after / total_rows_after):.{3}f}.")
print(f"Avg fill profit delta (after): {display['fill_profit_delta'].mean():.{2}f}")
print(f"Mode fill profit delta (before): {display['fill_profit_delta'].mode()[0]:.{2}f}")
print(f".75 Quantile fill profit delta (after): {display['fill_profit_delta'].quantile(0.25):.{2}f}")
print(f".80 Quantile fill profit delta (after): {display['fill_profit_delta'].quantile(0.20):.{2}f}")
print(f".85 Quantile fill profit delta (after): {display['fill_profit_delta'].quantile(0.15):.{2}f}")
print(f".90 Quantile fill profit delta (after): {display['fill_profit_delta'].quantile(0.10):.{2}f}")
print("-----------------------------------------------")
print_pretty(display)

Total Rows (before): 134
-----------------------------------------------
Num bad fills (before): 67
Avg fill profit delta (before): -45.34
Mode fill profit delta (before): 0.00
.75 Quantile fill profit delta (before): -75.00
.80 Quantile fill profit delta (before): -75.00
.85 Quantile fill profit delta (before): -100.00
.90 Quantile fill profit delta (before): -150.00
-----------------------------------------------
Total Rows (after): 134.
Num bad fills (after): 67
Bad fill percentage changed from 0.500 to 0.500.
Avg fill profit delta (after): -45.34
Mode fill profit delta (before): -0.00
.75 Quantile fill profit delta (after): -75.00
.80 Quantile fill profit delta (after): -75.00
.85 Quantile fill profit delta (after): -100.00
.90 Quantile fill profit delta (after): -150.00
-----------------------------------------------
set is ['fill_profit_delta']


ValueError: style is not supported for non-unique indices.

In [37]:
print(display)

run_id                    ok  ev_eid1_order  fill_ev_eid_delta  \
0        3  18446744073709551614           1849                  0   
1        2  18446744073709551614           1849                  0   
2        4  18446744073709551614           1849                  0   
3        1  18446744073709551614           1849                  0   
4        3  18446744073709551611           2374                  0   
5        2  18446744073709551611           2374                  0   
6        4  18446744073709551611           2374                  0   
7        1  18446744073709551611           2374                  0   
8        5  18446744073709551613          10193                  0   
9        7  18446744073709551613          10193                  0   
10       6  18446744073709551613          10193                  0   
11       8  18446744073709551613          10193                  0   
12       1  18446744073709551608          12555                  0   
13       3  184467440737

In [2]:
sweetness.style.set_properties(**{'text-align': 'right'})
#display = display.head(50)
sweetness.style.format({
   "fill_t_timestamp_delta": lambda t: f'{t.total_seconds():.2f}',
   "bid_p_0_avg_1s": "{:.2f}",
   "bid_p_0_avg_5s": "{:.2f}",
   "ask_p_0_avg_1s": "{:.2f}",
   "ask_p_0_avg_5s": "{:.2f}"
})

sweet_view = sweetness[["eid", "ev_eid1_order", "fill_profit_delta", "fill_t_timestamp_delta", "bid_p_0_avg_1s", "bid_p_0_avg_5s", "ask_p_0_avg_1s", "ask_p_0_avg_5s"]]
sweet_view.style.set_table_styles([
   dict(selector='td', props=[('text-align', 'right')])
   #dict(selector='tr', props=[('background', '#88ff0000')])
]).apply(simple_highlight_max, subset=['fill_profit_delta'])
sweet_view.sort_values('fill_profit_delta', ascending=True)
#display.head(2).to_html()

NameError: name 'sweetness' is not defined

In [499]:
sniper_run = pd.read_sql('select * from sniper_run', sim_db_con)
sniper_shots = pd.read_sql('select * from sniper_shots', sim_db_con)

In [500]:
clean_shots = sniper_shots[sniper_shots.clean == 1]
saved_shots = sniper_shots[sniper_shots.saved == 1]
forced_shots = sniper_shots[sniper_shots.forced == 1]

In [501]:
saved_shots['rgtm'] = 1
forced_shots['rgtm'] = -1

In [ ]:
sample_size = min(len(saved_shots), len(forced_shots))
saved_shots_sample = saved_shots.sample(sample_size)
forced_shots_sample = for

In [ ]:
stat_join = algo_order_forced_fills